In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [35]:
##Loading the dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


<h1>Preprocessing of the Data</h1>

In [36]:
#Drop irrelevant features
data = data.drop(["RowNumber", "CustomerId", "Surname"], axis =1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [37]:
#Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [38]:
#one hot encode on 'Geography' column
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[["Geography"]])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [39]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [40]:
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [41]:
#Combine one hot encoder columns  with the original data
data = pd.concat([data.drop('Geography', axis =1), geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [42]:
#save the encoders and scaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo,file)



In [43]:
#Divide the dataset into indepent and dependent features
X= data.drop('Exited', axis =1)
y = data['Exited']

#split the data in traing and testing data

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=42)
#scale these features
scaler = StandardScaler()
X_train =scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [44]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

ANN Implementation

In [45]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [46]:
#Building ANN model
model = Sequential([
    Dense(64,activation='relu', input_shape=(X_train.shape[1],)), #first hidden layer connected with input layer
    Dense(32, activation='relu'),#hl2
    Dense(1,activation='sigmoid')#output layer
])


/Users/dhaneshkumarkapadia/Desktop/LLMs/ANN/ANN-project-Implementation/venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [47]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate= 0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [49]:
#compile the model
model.compile(optimizer= opt, loss= loss, metrics= ['accuracy'])

In [50]:
#setup the tensorboard
log_dir = "logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [53]:
#setupEarly stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [54]:
#Trining pf the model
history = model.fit(
    X_train, y_train, validation_data=(X_test,y_test), epochs =100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step - accuracy: 0.8700 - loss: 0.3386 - val_accuracy: 0.8610 - val_loss: 0.3427
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 689us/step - accuracy: 0.8712 - loss: 0.3310 - val_accuracy: 0.8575 - val_loss: 0.3477
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - accuracy: 0.8717 - loss: 0.3260 - val_accuracy: 0.8655 - val_loss: 0.3373
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 673us/step - accuracy: 0.8629 - loss: 0.3287 - val_accuracy: 0.8605 - val_loss: 0.3399
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - accuracy: 0.8606 - loss: 0.3406 - val_accuracy: 0.8585 - val_loss: 0.3442
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 696us/step - accuracy: 0.8651 - loss: 0.3315 - val_accuracy: 0.8600 - val_loss: 0.3379
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 705us/step - accuracy: 0.8578 - loss: 0.3377 - val_accuracy: 0.8585 - val_loss: 0.3429
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 695us/step - accuracy: 0.8693 - loss: 0

In [55]:
model.save('modle.h5')

In [59]:
#load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [61]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 52906), started 0:02:23 ago. (Use '!kill 52906' to kill it.)

In [ ]:
#Load the pickle file
